In [ ]:
# 第3版 灰度重心法
import cv2
import numpy as np
import open3d as o3d
# 点云列表
point_data =[]
de =[]
dem =[]
n =751
for b in range(751):
    i_  = b*1000
    imageN = cv2.imread('./cap_v2/{}_diffN.bmp'.format(i_))
    imageP = cv2.imread('./cap_v2/{}_diffP.bmp'.format(i_)) 
    print(b)
    diff = cv2.subtract(imageP, imageN)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
    gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差
    m =[]
    for i in range(gray.shape[0]): #行数
        # 找到每一行最大亮度的索引
        #print(np.argmax(gray[i]),np.max(gray[i]))
        fenzi = fenmu = 0
        if np.max(gray[i])>5:  #10可以 5可以
            max_ind = np.argmax(gray[i])
            for j in range(max_ind-10,max_ind+10):
                fenzi = fenzi + j*gray[i][j]
                fenmu = fenmu + gray[i][j]
            center_ind = fenzi/fenmu #重心

            m.append(center_ind)
            # #四舍五入
            center_ind = round(center_ind)
            # 将最大值像素 置为255
            gray[i][center_ind]=255
            # 在原图上显示
            diff[i][center_ind]=[255, 255, 255] #白色

        #
            # 其余的像素置为0
            gray[i][:center_ind]=0
            gray[i][center_ind+1:]=0
            
        else:
            # 其余的像素置为0
            gray[i][:]=0
            m.append(np.inf)
    #cv2.imwrite("diff_{}.jpg".format(j), diff)
    baseline = b
    d =[]
    print(min(m)) 
    #print(len(m))
    #print(min(m)) #这里为1096
    for i in range(gray.shape[0]): #行数
        if np.max(gray[i]) != 0:
            deep =m[i] - (min(m))#1312   #1096#1096
            #print(m[i])
            #deep = min(max(np.argmax(gray[i])-1096,0),60) # 保持在-100到100之间
            if -4 <=(deep)<=70: #高度
                point_data.append([i,np.argmax(gray[i])-baseline,deep ])  #np.argmax(gray[i])
                d.append(deep)
        #gray[i][min(m)]=255 #显示用
    #print(d)
#     de.append(min(d))
#     dem.append(max(d))
# print(de)
# print(dem)
# print(min(de),max(dem))


#print(point_data)
# 将点数据转换为NumPy数组
points = np.array(point_data)
# 创建一个点云对象
pcd = o3d.geometry.PointCloud()
# 将NumPy数组中的点数据设置为点云的点
pcd.points = o3d.utility.Vector3dVector(points)
# 可视化点云
#显示坐标轴
#coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0]) #x红 y绿 z蓝
o3d.visualization.draw_geometries([pcd])
# 如果需要，可以将点云保存为PCD文件
o3d.io.write_point_cloud("output_v2_{}_z.pcd".format(n), pcd)

In [ ]:
import open3d as o3d
# 裁剪点云
pcd = o3d.io.read_point_cloud("output_v2_751_.pcd")
o3d.visualization.draw_geometries_with_editing([pcd])
 

# print("手动裁剪点云示例")

# print("按键 K 锁住点云，并进入裁剪模式")
# print("用鼠标左键拉一个矩形框选取点云，或者用 《ctrl+左键单击》 连线形成一个多边形区域")
# print("按键 C 结束裁剪并保存点云")
# print("按键 F 解除锁定，恢复自由查看点云模式")
# print("17873123809")

In [9]:
## 离群点移除
import open3d as o3d
n =751
pcd = o3d.io.read_point_cloud("cropped_z.ply")
o3d.visualization.draw_geometries([pcd])
print(len(pcd.points))

# # # 读取点云数据
# # pcd = o3d.io.read_point_cloud("path_to_your_point_cloud.pcd")

# 使用统计式离群点移除
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=600, std_ratio=1) # 50个点，标准差5
inlier_cloud = pcd.select_by_index(ind) # 选择内点
outlier_cloud = pcd.select_by_index(ind, invert=True) # 选择离群点

inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])  # 灰色表示内点
outlier_cloud.paint_uniform_color([1, 0, 0])       # 红色表示离群点
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
o3d.io.write_point_cloud("output_{}_statistical_z.ply".format(n), inlier_cloud)
print("离群点:",len(outlier_cloud.points))
print("内点:",len(inlier_cloud.points))

# # 使用半径式离群点剔除
# cl, ind = pcd.remove_radius_outlier(nb_points=50, radius=10) # 50个点，半径5
# inlier_cloud = pcd.select_by_index(ind)
# outlier_cloud = pcd.select_by_index(ind, invert=True)

# inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])  # 灰色表示内点
# outlier_cloud.paint_uniform_color([1, 0, 0])       # 红色表示离群点
# o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
# o3d.io.write_point_cloud("output_{}_半径式离群点移除_.pcd".format(n), inlier_cloud)
# print("离群点:",len(outlier_cloud.points))
# print("内点:",len(inlier_cloud.points))

562163
离群点: 30076
内点: 532087


In [12]:
# 曲率下采样 法向量空间下采样
import open3d as o3d
import numpy as np
olderr = np.seterr(all='ignore')


def vector_angle(x, y):
    lx = np.sqrt(x.dot(x))
    ly = (np.sum(y ** 2, axis=1)) ** 0.5
    cos_angle = np.sum(x * y, axis=1) / (lx * ly) 
    cos_angle = np.clip(cos_angle, -1, 1)  # 由于浮点数运算的精度问题，可能会出现略大于1或者略小于-1的情况
    angle = np.arccos(cos_angle)  # arccos计算出的角度是弧度制
    angle2 = np.degrees(angle)    # 阈值设置中用的是角度制，因此这里需要将弧度转换为角度 #[0,180]
    # 大于90 度的，被180度减去 
    angle2[angle2 > 90] = 180 - angle2[angle2 > 90] # [0,90]
    #print(angle2)
    return angle2


knn_num = 10     # 自定义参数值(邻域点数)
angle_thre = 10#30  # 自定义参数值(角度制度)
N = 5            # 自定义参数值(每N个点采样一次)
C = 5#10           # 自定义参数值(采样均匀性>N)

pcd = o3d.io.read_point_cloud("output_751_statistical_z.ply")
o3d.visualization.draw_geometries([pcd])
point = np.asarray(pcd.points)
point_size = point.shape[0] # 点的个数
tree = o3d.geometry.KDTreeFlann(pcd)  # 建立KD树索引 #KDTreeFlann表示快速最近邻搜索
o3d.geometry.PointCloud.estimate_normals(
    pcd, search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn_num))  # 计算法向量 #KDTreesearchParamKNN表示K近邻搜索
normal = np.asarray(pcd.normals) # 法向量
normal_angle = np.zeros(point_size)
for i in range(point_size):
    [_, idx, dis] = tree.search_knn_vector_3d(point[i], knn_num + 1) # 搜索knn_num个最近邻点 +1是因为包括自身
    current_normal = normal[i]
    knn_normal = normal[idx[1:]] #1:表示除了自身的其他点 #type为numpy.ndarray
    normal_angle[i] = np.mean(vector_angle(current_normal, knn_normal)) #
    #print(i,normal_angle[i])

point_high = point[np.where(normal_angle >= angle_thre)]  # 位于特征明显区域的点
point_low = point[np.where(normal_angle < angle_thre)]    # 位于特征不明显区域的点
pcd_high = o3d.geometry.PointCloud()
pcd_high.points = o3d.utility.Vector3dVector(point_high)
pcd_low = o3d.geometry.PointCloud()
pcd_low.points = o3d.utility.Vector3dVector(point_low)
#pcd_high_down = o3d.geometry.PointCloud.uniform_down_sample(pcd_high, N)  # 均匀采样特征明显区域
pcd_low_down = o3d.geometry.PointCloud.uniform_down_sample(pcd_low, C)    # 均匀采样特征不明显区域
pcd_finl = o3d.geometry.PointCloud()
pcd_finl.points = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd_high.points),   #
                                                             np.asarray(pcd_low_down.points))))  # 拼接点云
finl_point_size = np.asarray(pcd_finl.points).shape[0]
pcd_finl.paint_uniform_color([0.8, 0.8, 0.8]) # 灰色
print("原始点云点的个数为：", point_size)
print("下采样后点的个数为：", finl_point_size)
o3d.visualization.draw_geometries([pcd_finl], window_name="曲率下采样",
                                  width=1024, height=768,
                                  left=50, top=50,
                                  mesh_show_back_face=False)
o3d.io.write_point_cloud("output_751_down_z.ply", pcd_finl)

原始点云点的个数为： 532087
下采样后点的个数为： 288260


True

In [ ]:
# 高斯曲率2.0
import open3d as o3d
import numpy as np
from scipy.optimize import curve_fit

voxel_size = 1 #0.01越小密度越大
radius = 5#500#0.5   #搜索半径
num_knn = 100 #搜索邻域点的个数 #越多
pcd = o3d.io.read_point_cloud("output_751_down_z.ply") # "flaw.pcd" "cropped_2.ply"
print(len(pcd.points))
#pcd = pcd.voxel_down_sample(voxel_size) #下采样
o3d.visualization.draw_geometries([pcd])
print(len(pcd.points))
cloud = pcd
points = np.asarray(cloud.points) #点云转换为数组 点云数组形式为[[x1,y1,z1],[x2,y2,z2],...]
kdtree = o3d.geometry.KDTreeFlann(cloud) #建立KDTree
num_points = len(cloud.points) #点云中点的个数
pcd.paint_uniform_color([1, 0, 0])  # 初始化所有颜色为红色
# 定义非线性函数，这里假设是一个二次曲面
def func(x, a, b, c, d, e, f):
    return a * x[0]**2 + b * x[1]**2 + c * x[0] * x[1] + d * x[0] + e * x[1] + f
def f(x, y):
    return popt[0]*x**2 +popt[1]*y**2 +popt[2]* x*y +popt[3]*x + popt[4]*y +popt[5]
# 定义曲面的梯度函数，即一阶偏导数
def gradient(f, x, y):
    # 使用中心差分法近似求导  #参考https://cloud.tencent.com/developer/article/1685164
    h = 1e-6 # 差分步长，可以根据精度要求调整
    df_dx = (f(x + h, y) - f(x - h, y)) / (2 * h) # 对x求偏导
    df_dy = (f(x, y + h) - f(x, y - h)) / (2 * h) # 对y求偏导
    return df_dx, df_dy
# 定义曲面的曲率函数，即二阶偏导数
def curvature(f, x, y):
    # 使用中心差分法近似求导
    h = 1e-3 # 差分步长，可以根据精度要求调整 #越小越精确
    d2f_dx2 = (f(x + h, y) - 2 * f(x, y) + f(x - h, y)) / (h ** 2) # 对x求二阶偏导
    d2f_dy2 = (f(x, y + h) - 2 * f(x, y) + f(x, y - h)) / (h ** 2) # 对y求二阶偏导
    d2f_dxdy = (f(x + h, y + h) - f(x + h, y - h) - f(x - h, y + h) + f(x - h, y - h)) / (4 * h ** 2) # 对xy求混合偏导
    # 根据公式计算高斯曲率K和平均曲率H
    df_dx, df_dy = gradient(f, x, y) # 调用梯度函数求一阶偏导
    E = 1 + df_dx ** 2
    F = df_dx * df_dy
    G = 1 + df_dy ** 2
    L = d2f_dx2 / np.sqrt(1 + df_dx ** 2 + df_dy ** 2) #np.sqrt()表示开方
    M = d2f_dxdy / np.sqrt(1 + df_dx ** 2 + df_dy ** 2)
    N = d2f_dy2 / np.sqrt(1 + df_dx ** 2 + df_dy ** 2)
    K = (L * N - M ** 2) / (E * G - F ** 2) # 高斯曲率
    H = (E * N + G * L - 2 * F * M) / (2 * (E * G - F ** 2)) # 平均曲率
    return K, H

curvatures = []  
for i in range(num_points):
    #k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[i], radius) #返回邻域点的个数和索引
    k, idx, _ = kdtree.search_knn_vector_3d(cloud.points[i], num_knn) 
    #k, idx, _ = kdtree.search_hybrid_vector_3d(cloud.points[i],radius, num_knn) #返回半径内不超过num_knn个点的个数和索引
    neighbors = points[idx] #数组形式为[[x1,y1,z1],[x2,y2,z2],...]
    #print(k)
    Y = neighbors[:, 2] # 因变量
    X = neighbors[:, [0,1]] # 自变量 [[2,3],[1,1],[8,9],[11,12],[4,5],[8,9]] 6*2
    popt, pcov = curve_fit(func, xdata=X.T,ydata= Y)
    x = cloud.points[i][0] # 某一点的x坐标
    y = cloud.points[i][1] # 某一点的y坐标
    K, H = curvature(f, x, y) # 计算该点的曲率
    curvatures.append([K,H])

print(len(curvatures))
print(curvatures)
limit_max = 1e-3#1e-3#0#1e-8 ##1e-3
for i in range(len(curvatures)):
    if -limit_max<=curvatures[i][0] <= limit_max and -limit_max<=curvatures[i][1] <=limit_max: #平坦
        np.asarray(pcd.colors)[i] = [0, 0, 0]#黑
    elif -limit_max<=curvatures[i][0] <= limit_max and curvatures[i][1] >limit_max:  #凸
        np.asarray(pcd.colors)[i] = [1, 0, 0]#红
    elif -limit_max<=curvatures[i][0] <= limit_max and -limit_max<curvatures[i][1] <limit_max: #凹
        np.asarray(pcd.colors)[i] = [0, 1, 0]#绿
    elif curvatures[i][0] < -limit_max and curvatures[i][1] >limit_max: #鞍形脊 大部分凸，少部分凹
        np.asarray(pcd.colors)[i] = [0, 0, 1]#蓝
    elif curvatures[i][0] < -limit_max and curvatures[i][1] <-limit_max: #鞍形谷 大部分凹，少部分凸
        np.asarray(pcd.colors)[i] = [0, 1, 1]#青
    elif curvatures[i][0] > limit_max and curvatures[i][1] >limit_max: #峰
        np.asarray(pcd.colors)[i] = [1, 0, 1]#紫
    elif curvatures[i][0] > limit_max and curvatures[i][1] <-limit_max: #阱
        np.asarray(pcd.colors)[i] = [1, 1, 0]#黄


#显示点云
o3d.visualization.draw_geometries([pcd])

#保存点云
o3d.io.write_point_cloud("output_v2_曲率分类_.pcd", pcd)


 
